In [1]:
# install pycomm3 and ipywidgets packages
# !pip install -q pycomm3 ipywidgets

In [1]:
import pycomm3
from pycomm3 import logger
from pycomm3 import ClassCode, Services, ModuleIdentityObject, CIPDriver, data_types
import time

# logger = logger.configure_default_logger(level="VERBOSE", filename="pycomm3.log")

In [2]:
devices_list = pycomm3.CIPDriver.discover(broadcast_address="255.255.255.255")
print(devices_list)

[{'encap_protocol_version': 1, 'ip_address': '192.109.209.100', 'vendor': 'Cole-Parmer Instrument Company', 'product_type': 'Generic Device (keyable)', 'product_code': 1, 'revision': {'major': 3, 'minor': 2}, 'status': b'\x00\x00', 'serial': '7a65cb1d', 'product_name': 'Masterflex', 'state': 255}]


In [3]:
# Create a new CIPDriver object
pump_ip = "192.109.209.100"

In [14]:
response = None
with CIPDriver(pump_ip) as plc:
    response = plc.generic_message(
        service=Services.get_attributes_all,
        class_code=ClassCode.message_router,
        instance=b"\x01",
        connected=True,  # try with True as well
        #unconnected_send=False,  # set to True when connected is False and host has a route
                                                     # if host is just an IP, the omit all together
        name="identity",
    )
print(response.value)

b'\x07\x00\x01\x00\x02\x00\x04\x00\x06\x00H\x00\xf5\x00\xf6\x00\x1e\x00\x01\x00'


In [18]:
response = None
with CIPDriver(pump_ip) as plc:
    response = plc.generic_message(
        service=Services.get_attributes_all,
        class_code=b"\x07",
        instance=b"\x01",
        connected=True,  # try with True as well
        #unconnected_send=False,  # set to True when connected is False and host has a route
                                                     # if host is just an IP, the omit all together
        name="identity",
    )
print(response)

identity, b'\x07\x00\x0...e\x00\x01\x00', None, None


b'\x07\x00\x01\x00\x02\x00\x04\x00\x06\x00H\x00\xf5\x00\xf6\x00\x1e\x00\x01\x00'


In [6]:
def brute_force(service_code, class_code):
    response = None
    with CIPDriver(pump_ip) as plc:
        response = plc.generic_message(
            service=service_code,
            class_code=class_code,
            instance=b"\x70",
            attribute=b'\x03',
            connected=True,  # try with True as well
            #unconnected_send=False,  # set to True when connected is False and host has a route
                                                        # if host is just an IP, the omit all together
            name="test",
        )

    return response

# brute for all combination of the class code and instance code, record into a file
fix_class_code = ClassCode.assembly
for test_code in range(1, 0xFF):
        # put them into byte format
        print(f"COMMAND LINE OUTPUT=service_code={test_code}, class_code={fix_class_code}")
        response = brute_force(test_code, fix_class_code)
        with open("pycomm3.log", "a") as f:
            f.write(f"COMMAND LINE OUTPUT=service_code={test_code}, class_code={fix_class_code}, response={response}\n")
            time.sleep(5)
        print(f"COMMAND LINE OUTPUT=service_code={test_code}, class_code={fix_class_code}, response={response}")

COMMAND LINE OUTPUT=service_code=1, class_code=b'\x04'
COMMAND LINE OUTPUT=service_code=1, class_code=b'\x04', response=test, b'', None, Destination unknown, class unsupported, instance undefined or structure element undefined (see extended status) - Extended status out of memory  (05, 00)
COMMAND LINE OUTPUT=service_code=2, class_code=b'\x04'
COMMAND LINE OUTPUT=service_code=2, class_code=b'\x04', response=test, b'', None, Service not supported
COMMAND LINE OUTPUT=service_code=3, class_code=b'\x04'
COMMAND LINE OUTPUT=service_code=3, class_code=b'\x04', response=test, b'', None, Service not supported
COMMAND LINE OUTPUT=service_code=4, class_code=b'\x04'
COMMAND LINE OUTPUT=service_code=4, class_code=b'\x04', response=test, b'', None, Service not supported
COMMAND LINE OUTPUT=service_code=5, class_code=b'\x04'
COMMAND LINE OUTPUT=service_code=5, class_code=b'\x04', response=test, b'', None, Destination unknown, class unsupported, instance undefined or structure element undefined (see 

KeyboardInterrupt: 

In [ ]:
# original read tag example, this is using UDP
with pycomm3.CIPDriver(pump_ip) as drive:
    param = drive.generic_message(
        service=Services.get_attributes_all,
        class_code=ClassCode.assembly,
        instance=1,  # Parameter 41 = Accel Time
        attribute=b'\x09',
        data_type=data_types.INT,
        connected=False,
        unconnected_send=True,
        route_path=True,
        name='pf525_param'
    )
    print(param)

your_data = 500

# original write tag example, this is using UDP
with pycomm3.CIPDriver(pump_ip) as drive:
    drive.generic_message(
        service=Services.set_attribute_single,
        class_code=b'\x93',
        instance=41,  # Parameter 41 = Accel Time
        attribute=b'\x09',
        request_data=data_types.INT.encode(your_data),  # = 5 seconds * 100
        connected=False,
        unconnected_send=True,
        route_path=True,
        name='pf525_param'
    )

In [ ]:
# modified read tag example, UDP
#! unknown: class_code=b'\x93'
#! unknown: attribute
with pycomm3.CIPDriver(pump_ip) as pump:
    param = pump.generic_message(
        service=Services.get_attribute_single,
        class_code=ClassCode.parameter,
        instance=8,  # Parameter 8 = Remote Control
        # attribute=b'\x09', # omit for now
        data_type=data_types.BOOL,
        connected=False,
        unconnected_send=True,
        route_path=True,
        name='pf525_param'
    )
    print(param)

# modified write tag example using UDP
with pycomm3.CIPDriver(pump_ip) as pump:
    pump.generic_message(
        service=Services.set_attribute_single,
        class_code=ClassCode.parameter,
        instance=8,  # Parameter 8 = Remote Control
        # attribute=b'\x09', omit it for now
        request_data=data_types.BOOL.encode(1),  # = 5 seconds * 100
        connected=False,
        unconnected_send=True,
        route_path=True,
        name='pf525_param'
    )

In [9]:
import socket
import logging

# Setup logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class CIPClient:
    def __init__(self, ip, port, timeout=5):
        self.ip = ip
        self.port = port
        self.timeout = timeout
        self.sock = None

    def connect(self):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(self.timeout)
            logger.debug(f"Connecting to {self.ip}:{self.port}")
            self.sock.connect((self.ip, self.port))
            logger.debug("Connection established")
            return True
        except Exception as e:
            logger.error(f"Failed to connect to {self.ip}:{self.port} - {e}")
            return False

    def listen_for_reply(self):
        try:
            while True:
                data = self.sock.recv(1024)
                if not data:
                    break
                logger.debug(f"Received data: {data}")
                print(f"Received: {data}")
        except socket.timeout:
            logger.debug("Socket timeout, no more data.")
        except Exception as e:
            logger.error(f"Error receiving data - {e}")
        finally:
            self.close()

    def close(self):
        if self.sock:
            self.sock.close()
            logger.debug("Socket closed")

# get the current address of the computer
ip_address = socket.gethostbyname(socket.gethostname())
port = 44818  # Replace with the actual port number from the EDS file
client = CIPClient(ip_address, port)
if client.connect():
    client.listen_for_reply()

DEBUG:__main__:Connecting to 10.139.99.64:44818
ERROR:__main__:Failed to connect to 10.139.99.64:44818 - [WinError 10061] No connection could be made because the target machine actively refused it


In [3]:
class PumpController:
    """
    A PumpController class to handle communication with a pump using EtherNet/IP 
    protocol. It allows reading and writing parameters to the pump.
    """
    def __init__(self, ip_address):
        """
        Initializes the PumpController with the given IP address of the pump.

        Parameters:
        ip_address (str): The IP address of the pump.
        """
        self.ip_address = ip_address

        #! potential error here, don't how the tag list is initialized automatically or not
        #! or maybe the pump don't work with this package at all, then we have to fall back to the CIPDriver
        with pycomm3.LogixDriver(ip_address, init_tags=True) as pump:
            self.pump = pump
        #! potential error here, don't how the tag list is initialized automatically or not

        # I don't if open() is necessary here, official doc doesn't mention it
        open_return = self.pump.open()
        print(f"Open return: {open_return}")

        # Check if the connection was successful
        if self.pump.connected:
            print(f"Connected to pump at {ip_address}")

            # Print PLC info
            plc_info = self.pump.get_plc_info()

            print("PLC Info:")
            for key, value in plc_info.items():
                print(f"{key}: {value}")
        else:
            print(f"Failed to connect to pump at {ip_address}")

# if the device is not empty, we will attempt to create a PumpController object for each device
for device in devices_list:
    ip_address = device['ip_address']
    pump_controller = PumpController(ip_address)